In [1]:
import pandas as pd

## Utils and Library for notebook
from notebook_lib.nwpv.nwpv import nwpv_calculation
from notebook_utils.OpenKbcMSToolkit import ExtractionToolkit as exttoolkit

# Root data path
DATA_PATH = '../data/'

# Sample loading
gene_tpm = pd.read_feather(DATA_PATH+"counts_normalized/counts_vst_CD8.feather").set_index('index') # Load normalized CD8
meta_data = pd.read_csv(DATA_PATH+'EPIC_HCvB_metadata_baseline_updated-share.csv')

# Getting Sample Info
# DiseaseStatus: ['MS' 'Unknown' 'CIS' 'Healthy']
# DiseaseDuration(Early?)
# DiseaseCourse: ['RR' 'PP' 'SP' 'RIS' 'CIS' 'Unknown' 'Healthy']
sample_list, sample_category = exttoolkit.get_sample_name_by_category(dataframe=meta_data, sampleColumn='HCVB_ID', dataColname='DiseaseCourse')
print("Sample Count")
count=0 
for category, values in zip(sample_category, sample_list):
    print(category+" : "+str(len(values)) + ", List number : " + str(count))
    count+=1

Sample Count
RR : 82, List number : 0
PP : 14, List number : 1
SP : 1, List number : 2
RIS : 2, List number : 3
CIS : 40, List number : 4
Unknown : 1, List number : 5
Healthy : 22, List number : 6


In [2]:
duration_df = meta_data.dropna(subset=['DiseaseDuration']) # data for patient's duration set
duration_df['DiseaseDuration'] = duration_df['DiseaseDuration'].astype(float) # make float

## Long DD
longDD_samples = duration_df.loc[ duration_df['DiseaseDuration'] >= duration_df['DiseaseDuration'].median(), 'HCVB_ID'] # Get sampleData which has longDD from metadata
longDD_sampleList = list(set(gene_tpm.columns.tolist()).intersection(longDD_samples.values.tolist())) # Get intersected sampleID between expr and longDD
longDD_meta = duration_df[duration_df['HCVB_ID'].isin(longDD_sampleList)] # Get meta with sampleNames
longDD_gene_expr = gene_tpm[longDD_sampleList] # Get expr with sampleNames

## Short DD
shortDD_samples = duration_df.loc[ duration_df['DiseaseDuration'] < duration_df['DiseaseDuration'].median(), 'HCVB_ID']
shortDD_sampleList = list(set(gene_tpm.columns.tolist()).intersection(shortDD_samples.values.tolist()))
shortDD_meta = duration_df[duration_df['HCVB_ID'].isin(shortDD_samples.values.tolist())]
shortDD_gene_expr = gene_tpm[shortDD_sampleList]

/var/folders/sx/0rms4skn47nfn6svhhprv5700000gq/T/ipykernel_36505/3086605768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duration_df['DiseaseDuration'] = duration_df['DiseaseDuration'].astype(float) # make float


In [3]:
# NWPV calculation
#nwpv_class = nwpv_calculation(gene_tpm, shortDD_sampleList, longDD_sampleList)
#nwpv_class.get_result().to_csv("nwpv_result_CD8_vst.csv")
#nwpv_df = pd.read_csv('resultFiles/nwpv_result_CD8_vst.csv')
#nwpv_df[(nwpv_df['combined_pvalue_adj'] < 0.05)]

In [10]:
X = longDD_gene_expr.values[0]

72